In [6]:
from datasets import load_from_disk


auto_ravel = load_from_disk("./auto_ravel/Q5_processed")

In [19]:
new_attribute_types = []
from tqdm import tqdm

for tp in auto_ravel["attribute_type"]:
    if tp == "cause":
        new_attribute_types.append("causal")
    else:
        new_attribute_types.append(tp)
        
# auto_ravel is a Dataset object, replace the attribute_type column with the new attribute types
for i, tp in enumerate(tqdm(auto_ravel)):
    auto_ravel["attribute_type"][i] = tp



auto_ravel.save_to_disk("./auto_ravel/Q5_processed")

  0%|          | 0/35208 [00:00<?, ?it/s]

100%|██████████| 35208/35208 [07:27<00:00, 78.61it/s]


In [24]:
len(auto_ravel["attribute_type"]), len([a for a in auto_ravel["attribute_type"] if a == "cause"])



(35208, 3976)